In [1]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## 이미지 하나 Canny 적용해서 이미지 윤곽선, 배경 마스킹한 것 시각화하여 관찰

In [ ]:
# 준비한 영수증 이미지 경로를 넣기
image_path = '/data/ephemeral/home/data/chinese_receipt/img/train/extractor.zh.in_house.appen_000121_page0001.jpg'
image = cv2.imread(image_path)

# 그레이스케일로 변환
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Gaussian Blur로 노이즈 줄이기
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

#erosion_kernel = np.ones((10, 10), np.uint8)
#eroded_image = cv2.erode(blurred, erosion_kernel, iterations=1)
#_, binary_image = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# Canny edge detection 적용
edges = cv2.Canny(blurred, 75, 200)

# Apply dilation to thicken edges
kernel = np.ones((3, 3), np.uint8)
dilated_edges = cv2.dilate(edges, kernel, iterations=1)

# Find contours and filter to keep only the largest contour
contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)  # Sort contours by area
filtered_contours = [contours[0]] if contours else []  # Keep only the largest contour

# Create a mask and draw the largest contour on it
mask = np.zeros_like(gray)
if filtered_contours:
    cv2.drawContours(mask, filtered_contours, -1, 255, thickness=cv2.FILLED)

# Apply the mask to the original image to highlight the largest contour only
highlighted_image = cv2.bitwise_and(image, image, mask=mask)

# 각 단계별로 시각화
fig, axes = plt.subplots(1, 4, figsize=(20, 10))
axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axes[0].set_title("Original Image")
axes[0].axis("off")

axes[1].imshow(edges, cmap="gray")
axes[1].set_title("Canny Edges")
axes[1].axis("off")

axes[2].imshow(dilated_edges, cmap="gray")
axes[2].set_title("Dilated Edges")
axes[2].axis("off")

axes[3].imshow(cv2.cvtColor(highlighted_image, cv2.COLOR_BGR2RGB))
axes[3].set_title("Largest Contour Highlighted")
axes[3].axis("off")

plt.show()


## image_files 리스트에 있는 이미지 Canny 적용해서 이미지 윤곽선, 배경 마스킹하여 저장하기

In [23]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

# 중국 이미지 목록 설정
image_files = [
    "extractor.vi.in_house.appen_000852_page0001.jpg",
]


# 경로 설정
input_dir = '/data/ephemeral/home/Jeongseon/output/viet'  # 원본 이미지 경로
output_dir = '/data/ephemeral/home/Jeongseon/output/result/viet'  # 결과 저장 경로

# 결과 폴더 생성
os.makedirs(output_dir, exist_ok=True)

# 이미지 처리 함수
def process_and_save_image(image_path, output_path):
    # 이미지 로드 및 그레이스케일 변환
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Gaussian Blur 적용
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Canny edge detection 적용
    edges = cv2.Canny(blurred, 75, 200)

    # 엣지 확장
    kernel = np.ones((3, 3), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)

    # 가장 큰 윤곽만 남기기
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)  # 크기순으로 정렬
    filtered_contours = [contours[0]] if contours else []  # 가장 큰 윤곽만 사용

    # 마스크 생성 및 적용
    mask = np.zeros_like(gray)
    if filtered_contours:
        cv2.drawContours(mask, filtered_contours, -1, 255, thickness=cv2.FILLED)
    
    # 원본 이미지와 마스크 결합
    highlighted_image = cv2.bitwise_and(image, image, mask=mask)

    # 결과 이미지 저장
    cv2.imwrite(output_path, highlighted_image)
    print(f"Processed and saved: {output_path}")

# 각 이미지 처리 및 저장
for image_file in image_files:
    image_path = os.path.join(input_dir, image_file)
    output_path = os.path.join(output_dir, f"masked_{image_file}")
    
    if os.path.exists(image_path):
        process_and_save_image(image_path, output_path)
    else:
        print(f"Image not found: {image_path}")


Processed and saved: /data/ephemeral/home/Jeongseon/output/result/viet/masked_extractor.vi.in_house.appen_000852_page0001.jpg
